## Notebook generating figures for Harvey+25 Outshining Paper

In [ ]:
import numpy as np
from astropy.table import Table
from matplotlib import pyplot as plt
from matplotlib.ticker import ScalarFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cmasher as cmr
import matplotlib.cm as cm
import os
import glob
import astropy.units as u
import matplotlib.patheffects as pe
plt.rcParams["figure.dpi"] = 300
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
from EXPANSE import ResolvedGalaxy, ResolvedGalaxies
# import LogNorm

from matplotlib.colors import LogNorm

plt.style.use('/nvme/scratch/work/tharvey/scripts/paper_scientific.mplstyle')

Figure 1 showing the areas and depths for the JOF field is generated seperately by [GALFIND](https://galfind.readthedocs.io/) and not included here.



Figures 2 - curves of growth for the JOF field. Full code for PSFs is in the psfs/make_psf.py script.

In [ ]:
import sys
sys.path.append('/nvme/scratch/work/tharvey/EXPANSE/psfs')

from make_psf import psf_comparison, rel_cog_comparison

match_band = 'F444W'
surveys = ['JOF']
outdir_name = "JOF"

outdir = f"/nvme/scratch/work/tharvey/PSFs/{outdir_name}/"
outdir_webbpsf = f"/nvme/scratch/work/tharvey/PSFs/{outdir_name}/webbpsf/"
kernel_dir = f"/nvme/scratch/work/tharvey/PSFs/kernels/{outdir_name}/"

bands = [
    "F435W","F606W",'F775W',"F814W","F850LP","F090W",
    "F115W","F150W","F162M","F182M","F200W","F210M",
    "F250M","F277W","F300M","F335M","F356W","F410M","F444W",
    ]


psf_dir_dict = {
"+".join(surveys): outdir,
"UNCOVER DR3": "/nvme/scratch/work/tharvey/downloads/MEGASCIENCE_PSFs/",
"WebbPSF Default": f"{outdir_webbpsf}/default_jitter",
"WebbPSF\n$\\sigma=\\left\\{^{22(SW)}_{34(LW)}\\right.$ mas": f"{outdir_webbpsf}/morishita_jitter",
}
# Rederived 'NEW Webbpsf models have no difference to ours!
#'New webbpsf':'/nvme/scratch/work/tharvey/PSFs/webbpsf/'}
pixelscale = {
"+".join(surveys): 0.03,
"WebbPSF Default": 0.03,
"UNCOVER DR3": 0.04,
"WebbPSF\n$\\sigma=\\left\\{^{22(SW)}_{34(LW)}\\right.$ mas": 0.03,
}  # 'New webbpsf':0.03}


fig = psf_comparison(
    bands, psf_dir_dict, match_band=match_band, pixelscale=pixelscale
)

fig.savefig(f"/nvme/scratch/work/tharvey/EXPANSE/plots/final/psf_comparison_{outdir_name}.pdf", bbox_inches="tight", dpi=300)


Figure 3 - performance of pypher convolution kernels. 

In [ ]:
psf_dir_dict = {
    "+".join(surveys): outdir,
}

kernel_dir_dict = {
"+".join(surveys): kernel_dir,
}

fig = rel_cog_comparison(
    bands,
    psf_dir_dict,
    kernel_dir_dict,
    pixelscale=pixelscale,
    match_band=match_band,
)

fig.savefig(f"/nvme/scratch/work/tharvey/EXPANSE/plots/final/rel_cog_comparison_{outdir_name}.pdf", bbox_inches="tight", dpi=300)

Load EXPANSE Objects for JOF

In [ ]:
galaxies = ResolvedGalaxy.init_all_field_from_h5('JOF_psfmatched', '/nvme/scratch/work/tharvey/EXPANSE/galaxies/', n_jobs=6)

galaxies = ResolvedGalaxies(galaxies)

Figure 4 - The distribution of the sample in mass/redshift space, colored by effective radius.

In [ ]:
data = [[], [], []]
for galaxy in galaxies:
    try:
        r_eff = galaxy.pysersic_results['F444W_sersic_star_stack_sample']['meta']['results']['r_eff'][1] #pixels
        # Convert to kpc
        r_eff = r_eff * 0.03 * u.arcsec
        d_A = cosmo.angular_diameter_distance(galaxy.redshift).to(u.kpc).value
        r_eff = r_eff.to(u.rad).value * d_A
        mag = galaxy.total_photometry['F277W']['flux'] * u.uJy 
        mag = mag.to(u.ABmag).value
        redshift = galaxy.redshift
        data[0].append(redshift)
        data[1].append(mag)
        data[2].append(r_eff)
    except:
        pass


fig, ax = plt.subplots(figsize=(4, 4), dpi=300)
cax = make_axes_locatable(ax).append_axes('right', size='5%', pad=0.05)
sc = ax.scatter(data[0], data[1], c=data[2], cmap=cmr.guppy, norm=LogNorm(), s=20, edgecolor='black', linewidth=0.4)
cbar = fig.colorbar(sc, cax=cax, orientation='vertical')
cbar.set_label('Effective Radius (pkpc)')
ax.invert_yaxis()
ax.set_xlabel('Redshift')
ax.set_ylabel('F277W Total Mag [AB]')
cax.set_yticks([0.1, 0.2, 0.5, 1, 3, 5])

cbar.ax.yaxis.set_major_formatter(ScalarFormatter())

plt.savefig('/nvme/scratch/work/tharvey/EXPANSE/plots/final/mag_vs_redshift_v2.pdf', bbox_inches='tight')

Figure 5 - Example binning regions for the JOF field for different methodologies.

In [ ]:
galaxy_ids = ['10069', '12414', '12656', '15198', '16', '4040', '9196', '9634']

plot_galaxies = galaxies.filter_IDs(galaxy_ids)
ncols = 4
nrows = len(plot_galaxies)

# Calculate optimal figure size given the number of rows and columns, and the fact that all axis are square

figsize = (ncols * 1.5, nrows * 1.5)

# import make_axes_locatable
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, dpi = 200, tight_layout = True, facecolor = 'white', sharex=False, sharey=False)
# disable axis tick labels

for ax in axes.flatten():
    ax.axis('off')

# Label columns
for i, label in enumerate(['RGB', 'piXedfit', 'piXedfit (no min)', 'Voronoi']):
    axes[0, i].text(0.5, 1.05, label, transform=axes[0, i].transAxes, ha='center', va='bottom', fontsize=10)
     

for i, galaxy in enumerate(plot_galaxies):

    ax = axes[i, 0]
    rgb_stretch = 5e-4
    rgb_q = 5
    galaxy.plot_lupton_rgb(ax=ax, fig=fig,show=False, 
            red=["F444W"],
            green=["F356W"],
            blue=["F200W"],
            return_array=False,
            stretch=rgb_stretch,
            q=rgb_q,
            label_bands=True,
            add_compass=True if i == 0 else False,
            add_scalebar=True,
            compass_center = (17, 25),
            text_fontsize='x-small',
        )


    ax = axes[i, 1]

    map = galaxy.pixedfit_map
    mappable = ax.imshow(map, origin='lower', cmap='nipy_spectral_r', vmin=np.min(map), vmax=np.max(map))
    ax.text(0.05, 0.05, f'{galaxy.galaxy_id}\n z={galaxy.redshift:.2f}$^{{+{galaxy.meta_properties["zbest_84_fsps_larson_zfree"][0]-galaxy.meta_properties["zbest_fsps_larson_zfree"]:.2f}}}_{{-{galaxy.meta_properties["zbest_fsps_larson_zfree"]-galaxy.meta_properties["zbest_16_fsps_larson_zfree"][0]:.2f}}}$', transform=ax.transAxes, ha='left', va='bottom', fontsize=8,
              path_effects=[pe.withStroke(linewidth=1, foreground='white')])
    ax.text(0.05, 0.95, f'N$_{{\\rm{{bins}}}}$: {len(np.unique(galaxy.pixedfit_map))-1}  ', 
            transform=ax.transAxes, ha='left', va='top', fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')])
    #cbar = fig.colorbar(mappable, cax=cbar_ax)

    ax = axes[i, 2]
    #cbar_ax = make_axes_locatable(ax).append_axes('right', size='5%', pad=0.05)
    minmap = galaxy.pixedfit_nomin_map
    mappable = ax.imshow(minmap, origin='lower', cmap='nipy_spectral_r', vmin=np.min(minmap), vmax=np.max(minmap))
    # Label with ID and number of bins
    ax.text(0.05, 0.95, f'N$_{{\\rm{{bins}}}}$: {len(np.unique(galaxy.pixedfit_nomin_map))}', transform=ax.transAxes, ha='left', va='top', fontsize=8,
            path_effects=[pe.withStroke(linewidth=1, foreground='white')])
    #cbar = fig.colorbar(mappable, cax=cbar_ax)

    ax = axes[i, 3]

    try:
        #cbar_ax = make_axes_locatable(ax).append_axes('right', size='5%', pad=0.05)
        voronoi_map = galaxy.voronoi_map
        mappable = ax.imshow(voronoi_map, origin='lower', cmap='nipy_spectral_r', vmin=np.min(voronoi_map), vmax=np.max(voronoi_map))
        # Label with ID and number of bins
        ax.text(0.05, 0.95, f'N$_{{\\rm{{bins}}}}$: {len(np.unique(galaxy.voronoi_map))}', transform=ax.transAxes, ha='left', va='top', fontsize=8,
                path_effects=[pe.withStroke(linewidth=1, foreground='white')])
        #cbar = fig.colorbar(mappable, cax=cbar_ax)
    except AttributeError:
        # remove the axis
        ax.axis('off')

plt.subplots_adjust(wspace=-0.2)
plt.savefig('/nvme/scratch/work/tharvey/EXPANSE/plots/final/binning_methods.pdf', bbox_inches='tight')
plt.show()  


Figure 6 - Example of the SED fitting for a galaxy in the JOF field.

In [ ]:
galaxy = ResolvedGalaxy.init_from_h5("JOF_psfmatched_12880.h5")

fig = galaxy.plot_bagpipes_overview(pipes_run_dir='/nvme/scratch/work/tharvey/EXPANSE/pipes/', photometry_to_show=["TOTAL_BIN"], 
                            bagpipes_runs = {'CNST_SFH_RESOLVED_VORONOI':['RESOLVED'], 'photoz_delayed':['TOTAL_BIN']}, figsize=(12, 12), coloring_cmap='cmr.guppy',
                            binmap_type='voronoi', log_sfh=True, rgb_stretch=3e-4, rgb_q=3, rgb_combine_func=np.mean, 
                                    rgb_bands={'blue':['F090W', 'F115W', 'F150W', 'F162M', 'F182M'], 'green':['F356W', 'F277W', 'F300M', 'F335M'], 'red':['F410M', 'F444W']})
                                                
save_path = f'/nvme/scratch/work/tharvey/EXPANSE/plots/final/testing/JOF_psfmatched_{galaxy.galaxy_id}.pdf'

if not os.path.exists(os.path.dirname(save_path)):
    os.makedirs(os.path.dirname(save_path))

fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', pad_inches=1)
plt.show()

Figure 7 - Property Maps for a sample of galaxies.

In [ ]:
plot_galaxy_ids = ['12443', '893', '2779', '2171', '15589', '12541']
plot_galaxies = galaxies.filter_IDs(plot_galaxy_ids)

run_name = 'CNST_SFH_RESOLVED'
binmap_type = 'pixedfit'

for galaxy in plot_galaxies:
    try:
        if galaxy.redshift < 6.5:
            ew = 'Halpha_EWrest'
        else:
            ew = 'M_UV'
        fig = galaxy.plot_bagpipes_results(run_name=run_name, binmap_type=binmap_type, 
                        parameters=["lupton_rgb", "stellar_mass", "sfr","mass_weighted_age", "dust:Av",  "beta_C94", ew],
                        area_norm=True, weight_mass_sfr=True,  rgb_q=5, rgb_stretch=5e-4, scale_border=0.2, max_on_row=8,
                        override_param_names={'Halpha_EWrest': r'H\alpha \ EW \ ', 'OIII_5007_flux': r'[OIII]\AA \ Flux \ ', 
                                            'M_UV': r'M_{UV}', 'beta_C94': r'\beta_{C94} \ Slope', 'dust:Av': r' V-band \ Dust \ Attenuation ',
                                            'sfr_density':r'SFR\ density'})
        fig.savefig(f'/nvme/scratch/work/tharvey/EXPANSE/plots/final/testing/{galaxy.galaxy_id}_{run_name}.pdf', dpi = 200, bbox_inches='tight')
        
        plt.show()
    except Exception as e:
        #print(traceback.format_exc())

        print(e)

Figure 8 - Stellar mass vs. redshift with mass-density plots.

In [ ]:
run_name =  'CNST_SFH_RESOLVED'
binned_gals = galaxies.filter_single_bins('pixedfit')

fig = binned_gals.plot_resolved_map_scatter(binmap_type='pixedfit', 
                                      run_name=run_name, 
                                      map_param='stellar_mass',
                                      scale_pixel_pkpc=25,#200, 
                                      cmap='cmr.torch',#'RdYlBu',
                                      scale_physical=True,
                                      individual_cnorm=True,
                                      spread_factor=1.3,
                                      facecolor='black',
                                      norm_type='percentile',
                                      text_color='white', 
                                      scalebar_length=5,figsize=(25, 7),
                                      zoom_box = (5.6, 10.75, 6.3, 9.0), # (x1, x2, y1, y2) - box to zoom in on by zoom_factor
                                      zoom_factor = 2,
)
                                      

fig.savefig(f'/nvme/scratch/work/tharvey/EXPANSE/plots/final/{run_name}_resolved_map_scatter.pdf', dpi=300, bbox_inches='tight')


Figure 9 - Resolved stellar mass vs. integrated stellar mass.

In [ ]:
markers = ["o", "s", "D", "^", "v", "<", ">", "p", "P", "*", "X", "d", "H", "h", "+", "x", "|", "_", ".", ","]
bagpipes_runs = ["photoz_cnst", 'photoz_delayed', "photoz_lognorm", 'photoz_dblpwl', "photoz_continuity", "photoz_continuity_bursty",  'photoz_db'] # "photoz_delayed"
labels = ['Constant', r'Delayed-$\tau$', 'Lognormal', 'Double PL', 'Continuity',  'Continuity Bursty', 'Iyer et al. (2019)']
plt.style.use('/nvme/scratch/work/tharvey/scripts/paper.mplstyle')
fig, axs = plt.subplots(2, 2, figsize=(10, 10), dpi = 200, tight_layout=True, facecolor = 'white', sharex = True, sharey = True)
norm = plt.Normalize( vmin = 0, vmax = 12)
# make axes locatable
cmap = 'Spectral'
color_by = 'int_burstiness'
parameter = 'stellar_mass'

literature = Table.read('/nvme/scratch/work/tharvey/EXPANSE/data/OutshiningLiterature.csv', format='ascii.csv')
studies = np.unique(literature['Study'])
markers_lit = ['P', 'h', 'X', '*', 'd']

axs = axs.flatten()

cbar = fig.add_axes([1.01, 0.05, 0.02, 0.9])


cbar = fig.colorbar(
    cm.ScalarMappable(norm=norm, cmap=cmap),
    cax=cbar,
    orientation="vertical",
    label="Burstiness (SFR$_{10\mathrm{Myr}}$/SFR$_{100\mathrm{Myr}}$)"
)


resolved_runs = ['CNST_SFH_RESOLVED', 'CNST_SFH_RESOLVED_NOMIN', 'CNST_SFH_RESOLVED_VORONOI', 'CNST_SFH_RESOLVED_PBP']
resolved_maps = ['pixedfit', 'pixedfit_nomin', 'voronoi', 'pixel_by_pixel']
names = ['piXedfit', 'piXedfit \'nomin\'', 'Voronoi', 'Pixel by Pixel']

for pos, (resolved_run, resolved_map, ax, resolved_name) in enumerate(zip(resolved_runs, resolved_maps, axs, names)):
    for marker, bagpipes_run in zip(markers, bagpipes_runs):
        galaxies.comparison_plot(bagpipes_run, resolved_run, label = False, s = 15,
                                edgecolor = 'black', linewidths = 0.5, n_jobs = 1, cmap = cmap,
                                color_by=color_by, ax = ax, marker = marker, fig = fig, add_colorbar = False, norm = norm,
                                filter_single_bins_for_map = resolved_map, skip_missing = True, parameter=parameter, one_to_one=False)

    # Plot 0.5 dex offset line
    
    if 'sfr' in parameter:
        ax.set_xlim(1e-2, 3e2)
        ax.set_ylim(1e-2, 3e2)

    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.plot(xlim, np.array(ylim), color='black', linestyle='--', linewidth=1.5)

    
    # Label the line with angle text near the top of the line
    if parameter == 'stellar_mass':
        ax.plot(xlim, np.array(ylim)+0.5, color='black', linestyle='--', linewidth=1)
        
        ax.text(9.35, 9.95, r'0.5 dex', rotation=53, fontsize=8, color='black', ha = 'right', va = 'top', path_effects=[pe.withStroke(linewidth=2, foreground='white')])

        ax.plot(xlim, np.array(ylim)+1.0, color = 'black', linestyle = '--', linewidth = 1, alpha = 0.6)
        ax.text(8.85, 9.95, r'1.0 dex', rotation=53, fontsize=8, color='black', ha = 'right', va = 'top', path_effects=[pe.withStroke(linewidth=2, foreground='white')])
        

        #markers = ['o', 's', 'X', 'P', '*', 'v', '<', '>', 'p', 'P', '*', 'X', 'd', 'H', 'h', '+', 'x', '|', '_', '.']
        #colors = cm.tab20.colors
        
        lit_markers = []
        for i, study in enumerate(studies):
            mask = literature['Study'] == study
            if literature[mask]['ResolvedMassLowerErr'][0] > 0:
                ax.errorbar(literature[mask]['IntegratedMass'], literature[mask]['ResolvedMass'], xerr=[literature[mask]['IntegratedMassLowerErr'], literature[mask]['IntegratedMassUpperErr']], yerr=[literature[mask]['ResolvedMassLowerErr'], literature[mask]['ResolvedMassUpperErr']], label=study, marker='none', color='black', linestyle='None', capsize=0, markersize=5, zorder=1, alpha=0.5, linewidth=1)
            
            line = ax.scatter(literature[mask]['IntegratedMass'], literature[mask]['ResolvedMass'], label=study, marker=markers_lit[i], color='purple', s=45 if markers[i] in ['*', 'D'] else 30, zorder=2, alpha=1, edgecolor='black', linewidth=0.3)

            lit_markers.append(line)

        if pos == len(resolved_runs) - 1:
            lit_legend = ax.legend(handles = lit_markers, loc = 'lower right', fontsize = 8, title = 'Literature', markerscale = 2, bbox_to_anchor=(1, 0.07))
            
        if pos > 0:
            # Shade region above 9 on x-axis lightly
            ax.fill_between([9, 10], 0, 10, color = 'red', alpha = 0.1, ec='face', linewidth=0.00001)


    if pos == 0:
        # Fake a legend with the markers
        points = []
        for pos, (marker, bagpipes_run) in enumerate(zip(markers, bagpipes_runs)):
            point = ax.plot([], [], marker = marker, color = 'black', label = labels[pos])
            points.append(point[0])

        ax.legend(handles = points, loc = 'upper left', fontsize = 8, title = 'Integrated SFH')

    ax.text(0.98, 0.07, f"{resolved_name} Binning", transform=ax.transAxes, fontsize=15, ha='right', va='top', path_effects=[pe.withStroke(linewidth=2, foreground='white')], color='black')
    if 'sfr' in parameter:
        ax.set_xscale('log')
        ax.set_yscale('log')

text = 'SFR (10 Myr)'
text = 'Stellar Mass'

unit = 'M$_{\odot}$ yr$^{-1}$'
unit = '$\log_{{10}}(M_{{\odot}})$'

    
fig.supxlabel(rf'Integrated {text} [{unit}]', fontsize = 15, ha = 'center', va = 'center', y=0.02)
fig.supylabel(rf'Resolved {text} [{unit}]', fontsize = 15)
#get layout engine

fig.savefig(f'/nvme/scratch/work/tharvey/EXPANSE/plots/final/resolved_{parameter}_comparison_combined.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')

Figure 10 - Example SFHs for a sample of galaxies.

In [ ]:
ids = [10536, 9634, 10045, 15021, 5371]
ids = [str(i) for i in ids]

test_galaxies = galaxies.filter_IDs(ids)

galaxies_to_plot = test_galaxies 
fig = plt.figure(figsize=(4, 9), dpi = 200, tight_layout = True, facecolor = 'white')

axes = fig.subplots(len(galaxies_to_plot), 1, sharex=True)

resolved_runs = ['CNST_SFH_RESOLVED', 'CNST_SFH_RESOLVED_NOMIN', 'CNST_SFH_RESOLVED_VORONOI', 'CNST_SFH_RESOLVED_PBP']
resolved_run_names = ['Resolved (piXedfit)', 'Resolved (piXedfit, no min)', 'Resolved (voronoi)', 'Resolved (PBP)']
# nice color palette with distinct colors

resolved_cmap = 'cmr.apple'
resolved_colors = cmr.take_cmap_colors(resolved_cmap, len(resolved_runs), cmap_range=(0, 0.8))
resolved_colors = ['mediumseagreen', 'gold', 'orangered', 'royalblue']
resolved_color_dict = dict(zip(resolved_runs, resolved_colors))

integrated_runs = ['photoz_lognorm', 'photoz_delayed', 'photoz_continuity', 'photoz_cnst', 'photoz_continuity_bursty', 'photoz_dblpwl', 'photoz_db']
integrated_run_names = ['Lognorm SFH', r'Delayed-$\tau$ SFH', 'Continuity SFH', 'Constant SFH', 'Bursty SFH', 'Double PL SFH', 'Iyer+19 SFH']
# uniuqe cmap - not viridis
integrated_cmap = 'cmr.guppy'
# get colors from cmap
cmap = plt.get_cmap(integrated_cmap)
integrated_colors = [cmap(i) for i in np.linspace(0, 1, len(integrated_runs))]
integrated_color_dict = dict(zip(integrated_runs, integrated_colors))
resolved_run = "CNST_SFH_RESOLVED_VORONOI"
for i, galaxy in enumerate(galaxies_to_plot):
    ax = axes[i]

    for integrated_run, integrated_color in zip(integrated_runs, integrated_colors):
        _, _ = galaxy.plot_bagpipes_sfh(run_name = integrated_run, axes = ax, bins_to_show=['TOTAL_BIN'], fig = fig, marker_colors = [integrated_color], time_unit='yr', linestyle = 'dashdot', linewidth=1) #run_dir='../pipes/')
    
    
    for resolved_runa, resolved_color in zip(resolved_runs, resolved_colors):
        try:
            _, _ = galaxy.plot_bagpipes_sfh(run_name = resolved_runa, axes = ax, bins_to_show=['RESOLVED'], fig = fig, resolved_color=resolved_color, time_unit='yr', zorder=10, linewidth=1)#run_dir='../pipes/')
        except Exception as e:
            pass

    # Delete legend
    ax.get_legend().remove()

    ax.set_xscale('log')
    ax.set_xlabel('')
    # Label with galaxy ID and redshift

    ax.text(0.05, 0.95, f'{galaxy.galaxy_id}\nz={galaxy.redshift:.2f}\n$N_{{bins}}={galaxy.get_number_of_bins(binmap_type="voronoi")}$', transform=ax.transAxes, ha='left', va='top', zorder=30, fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')])
    
    # Write resolved stellar mass 

    # Get min, max mass from table
    min_mass = (1e10, 0, 0)
    max_mass = (0, 0, 0)
    for run in galaxy.sed_fitting_table['bagpipes'].keys():
        if run not in integrated_runs:
            continue
        
        if 'TOTAL_BIN' in galaxy.sed_fitting_table['bagpipes'][run]['#ID']:
            mass = galaxy.sed_fitting_table['bagpipes'][run]['stellar_mass_50'][0]
            upper = galaxy.sed_fitting_table['bagpipes'][run]['stellar_mass_84'][0] - mass
            lower = mass - galaxy.sed_fitting_table['bagpipes'][run]['stellar_mass_16'][0]

            if mass > max_mass[0]:
                max_mass = (mass, upper, lower, run)
            elif mass < min_mass[0]:
                min_mass = (mass, upper, lower, run)

    ax.text(0.97, 0.75, f'$\log_{{10}}(M_{{\star, \mathrm{{integrated, max}}}}) = {max_mass[0]:.2f}^{{+{max_mass[1]:.2f}}}_{{-{max_mass[2]:.2f}}}$', transform=ax.transAxes, ha='right', va='top', fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')], color=integrated_color_dict[max_mass[3]],  zorder=30)
    ax.text(0.97, 0.85, f'$\log_{{10}}(M_{{\star, \mathrm{{integrated, min}}}}) = {min_mass[0]:.2f}^{{+{min_mass[1]:.2f}}}_{{-{min_mass[2]:.2f}}}$', transform=ax.transAxes, ha='right', va='top', fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')], color=integrated_color_dict[min_mass[3]],  zorder=30)


    val = galaxy.resolved_mass[resolved_run][1]
    upper = galaxy.resolved_mass[resolved_run][2] - val
    lower = val - galaxy.resolved_mass[resolved_run][0]

    ax.text(0.97, 0.95, f'$\log_{{10}}(M_{{\star, \mathrm{{resolved}}}}) = {val:.2f}^{{+{upper:.2f}}}_{{-{lower:.2f}}}$', transform=ax.transAxes, ha='right', va='top', fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')], color=resolved_color_dict[resolved_run],  zorder=30)

    ax.set_xlim(1e6, 5e8)

# Add a dummy legend with SFHs

axes[-1].set_xlabel('Lookback Time [yr]')

points = []
for pos, (resolved_run, resolved_color) in enumerate(zip(resolved_run_names, resolved_colors)):
    point = ax.plot([], [], color = resolved_color, label = resolved_run)
    points.append(point[0])

for pos, (integrated_run, integrated_color) in enumerate(zip(integrated_run_names, integrated_colors)):
    point = ax.plot([], [], color = integrated_color, label = integrated_run, linestyle = 'dashdot')
    points.append(point[0])

fig.legend(handles = points, loc = 'lower center', fontsize = 8, title = 'SFH Type', title_fontsize = 8, markerscale = 0.7, labelspacing=0.1, ncol=2, bbox_to_anchor=(0.5, -0.11))


fig.savefig('/nvme/scratch/work/tharvey/EXPANSE/plots/final/sfh_comparison.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')


Figure 11 - Stellar mass discrepancy vs. integrated ssfr. 

In [ ]:
table = table = galaxies.save_to_fits(overwrite=True, save=False)

import statsmodels.api as sm
markers = ["o", "s", "D", "^", "v", "<", ">", "p", "P", "*", "X", "d", "H", "h", "+", "x", "|", "_", ".", ","]
markers = ["o", "s", "D", "^", "v", "<", ">", "p", "P", "*", "X", "d", "H", "h", "+", "x", "|", "_", ".", ","]
bagpipes_runs = ["photoz_cnst", 'photoz_delayed', "photoz_lognorm", 'photoz_dblpwl', "photoz_continuity", "photoz_continuity_bursty",  'photoz_db'] # "photoz_delayed"
labels = ['Constant', r'Delayed-$\tau$', 'Lognormal', 'Double PL', 'Continuity',  'Continuity Bursty', 'Iyer et al. (2019)']
plt.style.use('/nvme/scratch/work/tharvey/scripts/paper.mplstyle')
plot_dex_diff = True

delta_masses = {}
integrated = {}

colors = [
    "#D14D41",
    "#DA702C",
    #"#D0A215",
    "#879A39",
    "#3AA99F",
    "#4385BE",
    "#8B7EC8",
    "#CE5D97",
]

# Get colors from a colormap insead

cmap = 'cmr.tropical'
ncolors = len(bagpipes_runs)
colors = [plt.get_cmap(cmap)(1. * i / ncolors) for i in range(ncolors)]

resolved_runs = ['CNST_SFH_RESOLVED', 'CNST_SFH_RESOLVED_NOMIN', 'CNST_SFH_RESOLVED_VORONOI', 'CNST_SFH_RESOLVED_PBP']
rlabels = ['Constant SFH, piXedfit Binning', 'Constant SFH, piXedfit \'nomin\' Binning', 'Constant SFH, Voronoi Binning', 'Constant SFH, Pixel-by-Pixel Binning']
fig, axs = plt.subplots(len(resolved_runs), 2, figsize=(8, 3*len(resolved_runs)),
dpi = 200, tight_layout = True, facecolor = 'white', sharex='col', sharey='row', gridspec_kw={'wspace': 0}, width_ratios=[1, 0.2])


hist_axs = axs[:, 1]
axs = axs[:, 0]

hist_bins = np.arange(-0.8, 1.5, 0.2)

for rpos, (resolved_run, ax, rlabel) in enumerate(zip(resolved_runs, axs, rlabels)):
    counts = []
    for marker, bagpipes_run, color in zip(markers, bagpipes_runs, colors):
        delta_mass = table[f'{resolved_run}_resolved_mass'][:, 1] - table[f'{bagpipes_run}_stellar_mass_50']
        delta_masses[bagpipes_run] = delta_mass
        integrated[bagpipes_run] = table[f'{bagpipes_run}_stellar_mass_50']
        

        upper_error_resolved = table[f'{resolved_run}_resolved_mass'][:, 2] - table[f'{resolved_run}_resolved_mass'][:, 1]
        lower_error_resolved = table[f'{resolved_run}_resolved_mass'][:, 1] - table[f'{resolved_run}_resolved_mass'][:, 0]

        upper_error_bagpipes = table[f'{bagpipes_run}_stellar_mass_84'] - table[f'{bagpipes_run}_stellar_mass_50']
        lower_error_bagpipes = table[f'{bagpipes_run}_stellar_mass_50'] - table[f'{bagpipes_run}_stellar_mass_16']

        # Average the errors
        error_resolved = (upper_error_resolved + lower_error_resolved) / 2
        error_bagpipes = (upper_error_bagpipes + lower_error_bagpipes) / 2

        # Propogate errors
        error = np.sqrt(error_resolved**2 + error_bagpipes**2)

        yerr = error

        xerr_lower = table[f'{bagpipes_run}_ssfr_10myr_50'] - table[f'{bagpipes_run}_ssfr_10myr_16']
        xerr_upper = table[f'{bagpipes_run}_ssfr_10myr_84'] - table[f'{bagpipes_run}_ssfr_10myr_50']

        xerr = np.array([xerr_lower, xerr_upper])
        xerr[xerr < 0] = 0

        ax.errorbar(table[f'{bagpipes_run}_ssfr_10myr_50'], delta_mass, yerr = yerr, xerr=xerr, marker='none', linestyle='none', alpha = 0.3, zorder=2, color = color, linewidth = 1)
        ax.scatter(table[f'{bagpipes_run}_ssfr_10myr_50'], delta_mass, marker=marker, s=15, edgecolors='black', linewidth=0.5, zorder=3, color = color)

        # Add a LOWESS line
        x = np.array(list(table[f'{bagpipes_run}_ssfr_10myr_50']))
        y = np.array(list(delta_mass))

            # Medians
        bins=np.linspace(-10.0,-7.0,num=10)
        nbins=len(bins)
        delta = (bins[1]-bins[0])*0.5
        delta=0
        idx = np.digitize(x, bins)
        median = [np.nanmedian(y[idx==k]) for k in range(nbins)]
        # replace nans with 0
        #median = np.nan_to_num(median)
        ax.plot(bins-delta, median, c=color, alpha=0.5, lw=1.2, zorder=4, path_effects=[pe.withStroke(linewidth=2.6, foreground='white')])
   
        counts.append(delta_mass)
    
    #counts = np.array(counts).T
    # histogram on y-axis
    n, _, _ = hist_axs[rpos].hist(counts, bins=hist_bins, orientation='horizontal', color=colors, histtype='step', alpha=0.5)
    # Fix the x-axis limits
    ax.set_xlim(ax.get_xlim())
    ax.set_ylim(ax.get_ylim())

    # Make a dummy legend
    points = []
    for pos, (marker, bagpipes_run) in enumerate(zip(markers, bagpipes_runs)):
        point = ax.plot([], [], marker = marker, mec = 'black', label = labels[pos], color=colors[pos])
        points.append(point[0])

    ax.text(0.01, 0.96, rlabel, transform=ax.transAxes, fontsize=13, ha='left', va='top', path_effects=[pe.withStroke(linewidth=2, foreground='white')], color='black')
    if rpos == len(resolved_runs) - 1:
        ax.set_xlabel(r'Integrated Specific Star Formation Rate [10 Myr, $\log_{10}(\mathrm{yr}^{-1})$]', fontsize=11)
   
    ax.set_ylabel(r'Stellar Mass Offset [$\log_{10}(M_{\odot})$]', fontsize=11)
    # Plot 1:1, 0.5 and 1 dex lines
    xlim = ax.get_xlim()
    ax.plot(xlim, [0, 0], color='black', linestyle='--', linewidth=1)
    ax.plot(xlim, np.array([0, 0])+0.5, color='black', linestyle='--', linewidth=1)
    ax.plot(xlim, np.array([0, 0])+1.0, color = 'black', linestyle = '--', linewidth = 1, alpha = 0.6)
    
    if rpos == len(resolved_runs) - 1:
        leg = ax.legend(handles = points, loc = 'lower left', fontsize = 8, title = 'Integrated Bagpipes Run', frameon = True, title_fontsize = 8, markerscale = 0.7, labelspacing=0.1, fancybox = False, edgecolor = 'black')
            # change legend zorder
        for lh in leg.legend_handles: 
            lh.set_zorder(12)
    ax.set_xlim(-10, -6.8)
    ax.set_ylim(-0.5, 1.55)

    hist_axs[rpos].set_xlim(hist_axs[rpos].get_xlim())
    hist_axs[rpos].plot(hist_axs[rpos].get_ylim(), [0.0, 0.0], color='black', linestyle='--', linewidth=1)
    hist_axs[rpos].plot(hist_axs[rpos].get_ylim(), [0.5, 0.5], color='black', linestyle='--', linewidth=1)
    hist_axs[rpos].plot(hist_axs[rpos].get_ylim(), [1.0, 1.0], color='black', linestyle='--', linewidth=1, alpha=0.6)

    #ax.set_ylim(-0.6, 1.5)

    #ax.set_title(resolved_run)
    if rpos == 3:
        # plot sorba and sawiki 2018 piecewise fits
        def f(x, p, mu, B, k2):
            k1=  -2*k2*p + mu
            k0 = B + 12*mu + (mu-k1)*p-k2*p**2
            if x <= p:
                return mu*(x+12)+B
            else:
                return k2*x**2 + k1*x + k0

        p = -9.32
        mu = -0.02
        B = -0.024
        k2 = -0.21

        x = np.linspace(-10, -7.5, 100)

        y = [-f(i, p, mu, B, k2) for i in x]
        line = ax.plot(x, y, color='red', linestyle='solid', linewidth=3, label=r'Sorba & Sawicki (2018) [$z<2.5, \rm XDF~(z_{\rm s}+z_{\rm p})$]')

        x = np.linspace(-7.5, -7, 20)
        y = [-f(i, p, mu, B, k2) for i in x]
        ax.plot(x, y, color='red', linestyle='dotted', linewidth=3) #label='Sorba & Sawicki (2018) [$z<2.5, \rm XDF(z_{\rm s}+z_{\rm p})]')

        #add a legend just for this line

        ax.legend(handles = line, loc = 'upper left', fontsize = 8, title = 'Literature', frameon = True, title_fontsize = 8, markerscale = 0.7, labelspacing=0.1, fancybox = False, edgecolor = 'black', bbox_to_anchor=(0, 0.9))

        ax.add_artist(leg)
        


hist_axs[-1].set_xlabel('Counts')
hist_axs[0].set_xscale('log')
hist_axs[0].set_xlim(0.5, None)
# LinearTick labels

from matplotlib.ticker import ScalarFormatter
hist_axs[0].xaxis.set_major_formatter(ScalarFormatter())


fig.savefig('/nvme/scratch/work/tharvey/EXPANSE/plots/final/ssfr_vs_mass_offset.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')



Figure 12 - stellar mass discrepancy vs redshift.

In [ ]:
markers = ["o", "s", "D", "^", "v", "<", ">", "p", "P", "*", "X", "d", "H", "h", "+", "x", "|", "_", ".", ","]
bagpipes_runs = ["photoz_cnst", 'photoz_delayed', "photoz_lognorm", 'photoz_dblpwl', "photoz_continuity", "photoz_continuity_bursty",  'photoz_db'] # "photoz_delayed"
# generate a nice color for each run
colors = plt.get_cmap('tab20')(np.linspace(0, 1, len(bagpipes_runs)))


labels = ['Constant', r'Delayed-$\tau$', 'Lognormal', 'Double PL', 'Continuity',  'Continuity Bursty', 'Iyer et al. (2019)']
plt.style.use('/nvme/scratch/work/tharvey/scripts/paper.mplstyle')
fig, axs = plt.subplots(1, 2, figsize=(10, 5), dpi = 200, tight_layout=True, facecolor = 'white', sharex = True, sharey = True)
norm = plt.Normalize( vmin = 0, vmax = 12)
# make axes locatable
cmap = 'Spectral'
color_by = 'int_burstiness'
parameter = 'stellar_mass'


axs = axs.flatten()

cbar = fig.add_axes([1.01, 0.05, 0.02, 0.9])

cbar = fig.colorbar(
    cm.ScalarMappable(norm=norm, cmap=cmap),
    cax=cbar,
    orientation="vertical",
    label="Burstiness (SFR$_{10\mathrm{Myr}}$/SFR$_{100\mathrm{Myr}}$)"
)

import pandas as pd


resolved_runs = ['CNST_SFH_RESOLVED_VORONOI', 'CNST_SFH_RESOLVED_PBP'] # ['CNST_SFH_RESOLVED', 'CNST_SFH_RESOLVED_NOMIN', 
resolved_maps =  ['voronoi', 'pixel_by_pixel'] # ['pixedfit', 'pixedfit_nomin',
names = ['Voronoi', 'Pixel by Pixel'] #['piXedfit', 'piXedfit \'nomin\'', 

for pos, (resolved_run, resolved_map, ax, resolved_name) in enumerate(zip(resolved_runs, resolved_maps, axs, names)):
    for color, marker, bagpipes_run in zip(colors, markers, bagpipes_runs): 
        offset = table[f'{resolved_run}_resolved_mass'][:, 1] - table[f'{bagpipes_run}_stellar_mass_50']
        redshift = table['photoz_delayed_redshift_50']

        burstiness = table[f'{bagpipes_run}_sfr_10myr_50'] / table[f'{bagpipes_run}_sfr_50']

        
        ax.scatter(redshift, offset, label=labels, marker=marker, s=10, c=burstiness, cmap=cmap, norm=norm, alpha=0.7)

        x = np.array(redshift)
        y = np.array(offset)
        

        # Medians   
        if pos == 0:
            bins=np.arange(4, 11, 1)
        else:
            bins=np.arange(4, 10, 1)
        nbins=len(bins)
        delta = (bins[1]-bins[0])*0.5
        idx = np.digitize(x, bins)
        median = [np.nanmedian(y[idx==k]) for k in range(nbins)]
        # replace nans with 0
        #median = np.nan_to_num(median)
        
        ax.plot(bins-delta, median, c=color, alpha=0.9, lw=2, zorder=4, path_effects=[pe.withStroke(linewidth=3, foreground='white')])
                
    # Plot 0.5 dex offset line

    
    
    ax.text(0.97, 0.05, resolved_name, transform=ax.transAxes, fontsize=20, ha='right', va='bottom')

    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

# for each bagpipes run, plot marker and trend line in legend next to each other
markerso = []
lineso = []
for color, marker, bagpipes_run in zip(colors, markers, bagpipes_runs):
    m1 = ax.plot([], [], marker = marker, color = 'black', label = labels[pos], linestyle='none')
    l1 = ax.plot([], [], color = color, label = labels[pos], linestyle='-', linewidth=2, path_effects=[pe.withStroke(linewidth=3, foreground='black')], marker= 'none')
    markerso.append(m1[0])
    lineso.append(l1[0])
import matplotlib.legend_handler

handles = [(m, l) for m, l in zip(markerso, lineso)]

ax.legend(handles=handles, loc = 'upper left', fontsize = 8, title = 'Bagpipes Run', labels = labels, 
    handler_map={tuple: matplotlib.legend_handler.HandlerTuple(ndivide=None)}, frameon=True)


axs[0].set_ylabel(r'Stellar Mass Offset [$\log_{10}(M_{\odot})$]')

axs[0].set_xlabel(r'Photometric Redshift [z]')

axs[1].set_xlabel(r'Photometric Redshift [z]')

fig.savefig('/nvme/scratch/work/tharvey/EXPANSE/plots/final/resolved_mass_comparison_redshift.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')
    

Figure 13 -  Ha EW vs stellar mass discrepancy.

In [ ]:
redshifts = np.array([galaxy.sed_fitting_table['bagpipes']['photoz_delayed']['redshift_50'][0] for galaxy in galaxies])

galaxies_ha = galaxies[redshifts < 6.6]
ew = {}
for galaxy in galaxies_ha:
    map, kwargs = galaxy.galfind_phot_property_map("EW", frame='rest', rest_optical_wavs=[4_200.0, 10_000.0] * u.AA,
                                            strong_line_names = 'Halpha', n_jobs=1, plot = False, iters=100, load_in=False,
                                                z = 'photoz_delayed')
    if map is not None:
        #print(galaxy.photometry_properties['pixedfit']['EWrest_Halpha'])
        ew[galaxy.galaxy_id] = galaxy.photometry_properties['pixedfit']['EWrest_Halpha'][-6]



run_name = 'CNST_SFH_RESOLVED_VORONOI'
region = 'voronoi'
integrated_run = 'photoz_dblpwl'

fig, ax = plt.subplots(figsize=(6, 6), dpi=200)

delta_masses = {}
errors = {}
redshifts = {}
for galaxy in galaxies:
    try:
        new_mass = galaxy.get_total_resolved_property(run_name, 'stellar_mass', run_dir='/nvme/scratch/work/tharvey/EXPANSE/pipes/')
    except Exception as e:
        print(e)
        continue

    redshift = galaxy.sed_fitting_table['bagpipes'][run_name]['input_redshift'][0]

    if redshift > 6.6:
        continue

    int_table = galaxy.sed_fitting_table['bagpipes'][integrated_run]
    integrated_mass = int_table['stellar_mass_50'][0]
    #delta_mass = 10**new_mass[1] / 10**integrated_mass
    resolved_err = np.median([new_mass[1] - new_mass[0], new_mass[2] - new_mass[1]])
    integrated_err = np.median([int_table['stellar_mass_50'][0] - int_table['stellar_mass_16'][0], int_table['stellar_mass_84'][0] - int_table['stellar_mass_50'][0]])
    
    delta_mass = new_mass[1] - integrated_mass

    error = np.sqrt((resolved_err / new_mass[1])**2 + (integrated_err / integrated_mass)**2)

    delta_masses[galaxy.galaxy_id] = delta_mass
    errors[galaxy.galaxy_id] = error
    redshifts[galaxy.galaxy_id] = redshift
# Plot EW vs delta mass


cmap = cmr.redshift

norm = plt.Normalize(min(redshifts.values()), max(redshifts.values()))

for galaxy_id in ew:
    if galaxy_id not in delta_masses:
        continue
    ew_16, ew_50, ew_84 = np.percentile(ew[galaxy_id], [16, 50, 84])
    ew_gal = ew_50.value
    ew_16 = ew_16.value
    ew_84 = ew_84.value
    
    if ew_gal > 0:
        plt.scatter(ew_gal, delta_masses[galaxy_id], c=[cmap(norm(redshifts[galaxy_id]))], s=35, label=galaxy_id, edgecolors='black')

        plt.errorbar(ew_gal, delta_masses[galaxy_id], xerr=[[ew_gal - ew_16], [ew_84 - ew_gal]], fmt='none', capsize=0, alpha=0.4, lw=2,
                        color=cmap(norm(redshifts[galaxy_id])), yerr = errors[galaxy_id])
    else:
        plt.scatter(ew_84, delta_masses[galaxy_id], c=[cmap(norm(redshifts[galaxy_id]))], s=35, label=galaxy_id, edgecolors='black', marker='<')
   
cbar_ax = make_axes_locatable(ax).append_axes('right', size='5%', pad=0.05)
cbar = plt.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), cax=cbar_ax)
cbar.set_label('Redshift', fontsize=12)
ax.set_xlim(10, 6500)
ax.hlines(0, 0, ax.get_xlim()[1], linestyle='--', color='black', lw=1, alpha=0.8)
ax.set_xlabel(r'Inferred H$ \alpha$ EW [A]', fontsize=12)
ax.set_ylabel('Mass Offset [dex]', fontsize=12)
ax.set_xscale('log')

fig.savefig('/nvme/scratch/work/tharvey/EXPANSE/plots/final/ew_vs_mass_offset.pdf', dpi=300)


Figure 14 - Bar chart showing the closest integrated SFH to the resolved SFH for each galaxy.

In [ ]:
# Use table to work out which resolved SFH is closest to each resolved SFH


resolved_runs = ['CNST_SFH_RESOLVED', 'CNST_SFH_RESOLVED_NOMIN', 'CNST_SFH_RESOLVED_VORONOI', 'CNST_SFH_RESOLVED_PBP']
rlabels = ['piXedfit\nBinning', 'piXedfit \'nomin\'\nBinning', 'Voronoi\nBinning', 'Pixel-by-Pixel\nBinning']

bagpipes_runs = ["photoz_cnst", 'photoz_delayed', "photoz_lognorm", 'photoz_dblpwl', "photoz_continuity", "photoz_continuity_bursty",  'photoz_db'] # "photoz_delayed"
labels = ['Constant', r'Delayed-$\tau$', 'Lognormal', 'Double PL', 'Continuity',  'Continuity Bursty', 'Iyer et al. (2019)']

cmap = 'cmr.tropical'
ncolors = len(bagpipes_runs)
colors = [plt.get_cmap(cmap)(1. * i / ncolors) for i in range(ncolors)]


fig, axs = plt.subplots(1, 1, figsize=(6, 6), dpi = 200, tight_layout = True, facecolor = 'white')

width = 0.07

best_sfh_resolved = {}
for resolved_run in resolved_runs:
    best_sfh = {run:0 for run in bagpipes_runs}
    for pos, row in enumerate(table):
        best_mass = np.inf
        best_sfh_run = ''
        for bagpipes_run in bagpipes_runs:
            if table[f'{resolved_run}_resolved_mass'].mask[pos, 1]:
                #print(row[f'{resolved_run}_resolved_mass'][1])
                continue
            
            delta_mass = row[f'{resolved_run}_resolved_mass'][1] - row[f'{bagpipes_run}_stellar_mass_50']
            if np.abs(delta_mass) < best_mass:
                best_mass = np.abs(delta_mass)
                best_sfh_run = bagpipes_run
        if best_sfh_run == '':
            pass
            #print('No best SFH found')
        else:
            best_sfh[best_sfh_run] += 1
    best_sfh_resolved[resolved_run] = best_sfh

# Make a bar chart, grouped by each resolved SFH

norms =np.array([np.sum(list(best_sfh.values())) for best_sfh in best_sfh_resolved.values()])

for pos, bagpipes_run in enumerate(bagpipes_runs):
    x = np.arange(len(resolved_runs))
    y = [best_sfh_resolved[resolved_run][bagpipes_run] for resolved_run in resolved_runs]

    y = 100 * np.array(y) / norms
    axs.bar(x + pos*(width+0.01), y, width, label=labels[pos], color=colors[pos])

        
axs.set_xticks(x + width * len(bagpipes_runs) / 2)
axs.set_xticklabels(rlabels)

# Disable grid
axs.grid(False)

axs.legend(title='Integrated SFH', loc='upper left', fontsize=10, title_fontsize=13, markerscale=0.7, labelspacing=0.1, fancybox=False, edgecolor='black')
axs.set_ylabel(r'Fraction of Galaxies [%]')
axs.set_xlabel('Binning Method')

# Add an arrow showing N_bins/gal from left to right

axs.annotate('', xy=(0.6, 0.88), xytext=(0.4, 0.88), arrowprops=dict(facecolor='black', arrowstyle='-|>'), xycoords='axes fraction', textcoords='axes fraction')
axs.text(0.5, 0.9, r'N$_{\rm bins}$/galaxy', ha='center', va='center', fontsize=10, color='black', path_effects=[pe.withStroke(linewidth=2, foreground='white')], transform=axs.transAxes)

fig.savefig('/nvme/scratch/work/tharvey/EXPANSE/plots/final/best_sfh_resolved.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')

Figure 15 - Resolved Bursty SFH vs Resolved Constant SFH.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 4), dpi=200, tight_layout=True)

runs_x = ['CNST_SFH_RESOLVED', 'CNST_SFH_RESOLVED_NOMIN', 'CNST_SFH_RESOLVED_VORONOI']
runs_y = ['BURSTY_SFH_RESOLVED', 'BURSTY_SFH_RESOLVED_NOMIN', 'BURSTY_SFH_RESOLVED_VORONOI']
titles = ['piXedfit', 'piXedfit No Min', 'Voronoi']

norm = cm.colors.Normalize(vmin=4.5, vmax=11.5)
cmap = cm.plasma

offsets_run = {0:[], 1:[], 2:[]}
for galaxy in galaxies:

    for i,(run_x, run_y)in enumerate(zip(runs_x, runs_y)):
        try:
            old_mass = galaxy.get_total_resolved_property(run_x, 'stellar_mass')
            new_mass = galaxy.get_total_resolved_property(run_y, 'stellar_mass')

            redshift = galaxy.sed_fitting_table['bagpipes'][run_x]['input_redshift'][0]
            ax[i].scatter(new_mass[1], old_mass[1], label=f'{run_x} vs {run_y}', s = 10, color=cmap(norm(redshift)))
            ax[i].errorbar(new_mass[1], old_mass[1], xerr = [[new_mass[1]-new_mass[0]], [new_mass[2]-new_mass[1]]], yerr = [[old_mass[1]-old_mass[0]], [old_mass[2]-old_mass[1]]],
                            linestyle='None', marker='None', alpha=0.4, color=cmap(norm(redshift)))
            offsets = old_mass[1] - new_mass[1]
            offsets_run[i].append(offsets)
            
        except Exception as e:
            print(e)
            pass

       
        ax[i].set_xlim([6.5, 9])
        ax[i].set_ylim([6.5, 9])
        ax[i].plot([6.5, 11], [6.5, 11], color='black', linestyle='--', lw=1, alpha=0.6)
        ax[i].set_xlabel('Resolved Stellar Mass [Bursty SFH]', fontsize=10)
        ax[i].set_ylabel('Resolved Stellar Mass [Constant SFH]', fontsize=10)
        ax[i].text(0.02, 0.98, f'{titles[i]} Binning', transform=ax[i].transAxes, fontsize=12, verticalalignment='top', horizontalalignment='left', color='black', alpha=0.8, path_effects=[pe.withStroke(linewidth=2, foreground='white')])

for i in range(3):
    ax[i].text(0.02, 0.93, f'Median $\Delta$M$_\star$: {np.nanmedian(offsets_run[i]):.2f}', transform=ax[i].transAxes, fontsize=9, verticalalignment='top', horizontalalignment='left', color='black', alpha=0.8, path_effects=[pe.withStroke(linewidth=2, foreground='white')])
    ax[i].text(0.02, 0.88, f'Mean $\Delta$M$_\star$: {np.nanmean(offsets_run[i]):.2f}', transform=ax[i].transAxes, fontsize=9, verticalalignment='top', horizontalalignment='left', color='black', alpha=0.8, path_effects=[pe.withStroke(linewidth=2, foreground='white')])

cbar_ax = fig.add_axes([0.952, 0.16, 0.02, 0.6])
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax, label='Redshift')
# Change labels to other side
cbar_ax.yaxis.set_label_position('left')
cbar_ax.yaxis.set_ticks_position('left')

fig.savefig('/nvme/scratch/work/tharvey/EXPANSE/plots/final/bursty_vs_constant_mass.pdf', dpi=300)


Figure 16 - Dense Basis SED fitting results.

In [ ]:
binmap = 'voronoi'
db_atlas_path = f'/nvme/scratch/work/tharvey/EXPANSE/scripts/pregrids/db_atlas_JOF_500000_Nparam_3.dbatlas'
from EXPANSE.dense_basis import get_priors
priors = get_priors(db_atlas_path)

plot_galaxies = galaxies.filter_single_bins(binmap)

fig = plt.figure(figsize=(8, 5), dpi=200, tight_layout=True, facecolor='white')

# Make a gridspec with one large square plot on the left and three smaller rectangular plots on the right

from matplotlib.gridspec import GridSpec

nrows = 4
gs = GridSpec(nrows, 2, figure=fig)

ax = fig.add_subplot(gs[:, 0])

sfh_axs = [fig.add_subplot(gs[i, 1]) for i in range(nrows)]


# Add ticks on right and top
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.minorticks_on()
ax.tick_params(axis='both', direction='in', which='both')

cax = make_axes_locatable(ax).append_axes('top', size='5%', pad=0.05)

cmap = cmr.guppy
norm = plt.Normalize(4.5, 12)

data = {}
for galaxy in plot_galaxies:
    try:
        resolved_mass = galaxy.get_total_resolved_property(f'db_atlas_JOF_star_stack_{binmap}_zphotoz_delayed', 
                                            sed_fitting_tool='dense_basis',
                                            property='mstar')
        resolved_mass_16 = galaxy.get_total_resolved_property(f'db_atlas_JOF_star_stack_{binmap}_zphotoz_delayed', 
                                            sed_fitting_tool='dense_basis',
                                            property='mstar_16')
        resolved_mass_84 = galaxy.get_total_resolved_property(f'db_atlas_JOF_star_stack_{binmap}_zphotoz_delayed', 
                            sed_fitting_tool='dense_basis',
                            property='mstar_84')

        table = galaxy.sed_fitting_table['dense_basis'][f'db_atlas_JOF_star_stack_{binmap}_zphotoz_delayed']
        mask = table['#ID'] == 'TOTAL_BIN'
        integrated_mass = table['mstar_50'][mask][0]
        lower_error = integrated_mass - table['mstar_16'][mask][0]
        upper_error = table['mstar_84'][mask][0] - integrated_mass
        redshift = galaxy.redshift

        upper_resolved_error = resolved_mass_84 - resolved_mass
        lower_resolved_error = resolved_mass - resolved_mass_16

        data[galaxy.galaxy_id] = [integrated_mass, resolved_mass, [lower_error, upper_error], [lower_resolved_error, upper_resolved_error]]
        #
        #if integrated_mass < 7.6:
        #    ax.text(integrated_mass, resolved_mass-0.05, galaxy.galaxy_id, fontsize=6, ha='center', va='center', color='black', zorder=5, path_effects=[pe.withStroke(linewidth=1, foreground='white')])
        ax.scatter(integrated_mass, resolved_mass, color=cmap(norm(redshift)), label=galaxy.galaxy_id, zorder=4, s=10, edgecolors='black', linewidths=1)
        ax.errorbar(integrated_mass, resolved_mass, xerr=[[lower_error], [upper_error]], yerr = [[lower_resolved_error], [upper_resolved_error]],
                     marker='none', alpha=0.3, color=cmap(norm(redshift)), zorder=2, linewidth=1)
    except Exception as e:
        print(e)
        print(galaxy.galaxy_id)
        print('crash!')
        pass

# 2439, 8628, 15156, 3217, 10045
overwrite = False
ids_to_plot_sfh = [3217, 10489, 8628, 15156]
colors = ['blue', 'green', 'red', 'purple']
filter_galaxies = galaxies.filter_IDs([str(i) for i in ids_to_plot_sfh])
for i, galaxy in enumerate(filter_galaxies):
    # pair of nice contrasting colors for SFH
    nice_colors = ['dodgerblue', 'orangered']
    #overwrite = True if galaxy.galaxy_id == '8628' else False
    if overwrite or galaxy.sed_fitting_sfhs is None or f'db_atlas_JOF_star_stack_{binmap}_zphotoz_delayed' not in galaxy.sed_fitting_sfhs['dense_basis'].keys():
            fit_results = galaxy.run_dense_basis(db_atlas_path, plot=False, fit_photometry='TOTAL_BIN+bin',
                                        fix_redshift='photoz_delayed', binmap_type=binmap, use_emcee=False,
                                        priors = priors, n_jobs=1, save_outputs=True, save_sfh=True, overwrite=True,
                                        save_full_posteriors=False, parameters_to_save=['mstar','zval'])
    galaxy.plot_internal_sfh(ax=sfh_axs[i], sed_fitter='dense_basis',
                            run_name = 'db_atlas_JOF_star_stack_voronoi_zphotoz_delayed',
                            bins_to_show=['RESOLVED', 'TOTAL_BIN'], colors=nice_colors)
    if i == 0:
        handles, labels = sfh_axs[i].get_legend_handles_labels()
        # replace 'TOTAL_BIN' with 'Integrated'
        labels = [label.replace('TOTAL_BIN', 'Integrated') for label in labels]
        labels = [label.replace('RESOLVED', 'Resolved') for label in labels]

        leg = sfh_axs[i].legend(loc='center', fontsize=7, title='SFH Type', title_fontsize=9, markerscale=0.7, labelspacing=0.1, handles = handles, labels = labels, bbox_to_anchor=(0.5, 1.30))
    if i != len(filter_galaxies) - 1:
        sfh_axs[i].set_xlabel('')
    # Label with redshift and galaxy ID
    mass = data[galaxy.galaxy_id][1]
    error= data[galaxy.galaxy_id][3]
    sfh_axs[i].text(0.01, 0.965, f'{galaxy.galaxy_id} | z={galaxy.redshift:.2f} | $N_{{bins}}={galaxy.get_number_of_bins(binmap_type="voronoi")}$ | M$_{{\star, res}} ={mass:.2f}^{{+{error[1]:.2f}}}_{{-{error[0]:.2f}}}$', transform=sfh_axs[i].transAxes, ha='left', va='top', zorder=30, fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')], color=colors[i],
                    bbox=dict(facecolor='white', edgecolor='black', boxstyle='square,pad=0.3'))
    # Connect point on main plot with SFH plot - put a box around the point
    ax.scatter(data[galaxy.galaxy_id][0], data[galaxy.galaxy_id][1], zorder=5, s=40, edgecolors=colors[i], linewidths=1.5, facecolors='none', marker = 's')

    sfh_axs[i].yaxis.set_ticks_position('both')
    sfh_axs[i].xaxis.set_ticks_position('both')
    sfh_axs[i].minorticks_on()
    sfh_axs[i].tick_params(axis='both', direction='in', which='both')
    sfh_axs[i].set_ylabel('')

# add a 1:1 line

fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax, orientation='horizontal', label='Redshift')
cax.xaxis.set_ticks_position('top')
cax.xaxis.set_label_position('top')

ax.plot([7, 11], [7, 11], color='black', linestyle='--', linewidth=1)
ax.set_xlim(7, 11)
ax.set_ylim(7, 11)

ax.plot([7, 11], [7.5, 11.5], color='black', linestyle='dotted', linewidth=1)
ax.plot([7, 11], [8, 12], color='black', linestyle='dotted', linewidth=1)

fig.text(0.53, 0.5, 'Star Formation Rate [M$_\odot$ yr$^{-1}$]', va='center', ha='center', rotation='vertical')

ax.set_xlabel(r'Integrated Mass [$\log_{{10}}(M_\star/ M_\odot)$]')
ax.set_ylabel(r'Resolved Mass [$\log_{{10}}(M_\star/ M_\odot)$]')

ax.text(0.05, 0.95, 'Dense Basis SED Fitting', va='top', ha='left', fontsize=10, color='black', transform=ax.transAxes)
ax.text(0.05, 0.90, f'{binmap.capitalize()} binning', va='top', ha='left', fontsize=10, color='black', transform=ax.transAxes)
fig.savefig(f'/nvme/scratch/work/tharvey/EXPANSE/plots/final/mass_comparison_db_{binmap}.pdf', dpi=300)

Figure 17 - GSMF with outshining correction is generated by a seperate code written for the Harvey+25 GSMF paper and is not included here. Please get in touch with the authors if you're interested in this code.

Figure A1 - is screenshots of the SED fitting interface. If you install the EXPANSE package you can launch the interface using 'expanse-viewer' in the terminal. We have provided examples of how to create your own ResolvedGalaxy object to load in the examples/ directory.

Figure C1 - Resolved SFR vs integrated SFR.

In [ ]:
markers = ["o", "s", "D", "^", "v", "<", ">", "p", "P", "*", "X", "d", "H", "h", "+", "x", "|", "_", ".", ","]
bagpipes_runs = ["photoz_cnst", 'photoz_delayed', "photoz_lognorm", 'photoz_dblpwl', "photoz_continuity", "photoz_continuity_bursty",  'photoz_db'] # "photoz_delayed"
labels = ['Constant', r'Delayed-$\tau$', 'Lognormal', 'Double PL', 'Continuity',  'Continuity Bursty', 'Iyer et al. (2019)']
plt.style.use('/nvme/scratch/work/tharvey/scripts/paper.mplstyle')
fig, axs = plt.subplots(2, 2, figsize=(10, 10), dpi = 200, tight_layout=True, facecolor = 'white', sharex = True, sharey = True)
norm = plt.Normalize( vmin = 0, vmax = 12)
# make axes locatable
cmap = 'Spectral'
color_by = 'int_burstiness'
parameter = 'sfr'


axs = axs.flatten()

cbar = fig.add_axes([1.01, 0.05, 0.02, 0.9])


cbar = fig.colorbar(
    cm.ScalarMappable(norm=norm, cmap=cmap),
    cax=cbar,
    orientation="vertical",
    label="Burstiness (SFR$_{10\mathrm{Myr}}$/SFR$_{100\mathrm{Myr}}$)"
)


resolved_runs = ['CNST_SFH_RESOLVED', 'CNST_SFH_RESOLVED_NOMIN', 'CNST_SFH_RESOLVED_VORONOI', 'CNST_SFH_RESOLVED_PBP']
resolved_maps = ['pixedfit', 'pixedfit_nomin', 'voronoi', 'pixel_by_pixel']
names = ['piXedfit', 'piXedfit \'nomin\'', 'Voronoi', 'Pixel by Pixel']

for pos, (resolved_run, resolved_map, ax, resolved_name) in enumerate(zip(resolved_runs, resolved_maps, axs, names)):
    for marker, bagpipes_run in zip(markers, bagpipes_runs):
        galaxies.comparison_plot(bagpipes_run, resolved_run, label = False, s = 15,
                                edgecolor = 'black', linewidths = 0.5, n_jobs = 1, cmap = cmap,
                                color_by=color_by, ax = ax, marker = marker, fig = fig, add_colorbar = False, norm = norm,
                                filter_single_bins_for_map = resolved_map, skip_missing = True, parameter=parameter, one_to_one=False)

    # Plot 0.5 dex offset line
    
    ax.set_xlim(1e-2, 3e2)
    ax.set_ylim(1e-2, 3e2)

    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.plot(xlim, np.array(ylim), color='black', linestyle='--', linewidth=1.5)

    if pos == 0:
        # Fake a legend with the markers
        points = []
        for pos, (marker, bagpipes_run) in enumerate(zip(markers, bagpipes_runs)):
            point = ax.plot([], [], marker = marker, color = 'black', label = labels[pos])
            points.append(point[0])

        ax.legend(handles = points, loc = 'upper left', fontsize = 8, title = 'Integrated SFH', fancybox = False)

    ax.text(0.98, 0.07, f"{resolved_name} Binning", transform=ax.transAxes, fontsize=15, ha='right', va='top', path_effects=[pe.withStroke(linewidth=2, foreground='white')], color='black')
    if 'sfr' in parameter:
        ax.set_xscale('log')
        ax.set_yscale('log')

text = 'SFR'

unit = 'M$_{\odot}$ yr$^{-1}$'

    
fig.supxlabel(rf'Integrated {text} [{unit}]', fontsize = 15, ha = 'center', va = 'center', y=0.02)
fig.supylabel(rf'Resolved {text} [{unit}]', fontsize = 15)
#get layout engine

fig.savefig(f'/nvme/scratch/work/tharvey/EXPANSE/plots/final/resolved_{parameter}_comparison_combined.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')